In [49]:
# write function that takes a path of a directory : returns a list of all the paths of the files in that directory
import os
def getFiles(path):
    files = []
    for r, d, f in os.walk(path):
        for file in f:
            files.append(os.path.join(r, file))
    return files
print(getFiles("./data"))

['./data/NOAM CHOMSKY - Defining our times existential risks, propaganda, war, hope and the meaning of life [English (auto-generated)] [DownloadYoutubeSubtitles.com].txt', './data/Beyond the darkest hour OTTO SCHARMER [English (auto-generated)] [DownloadYoutubeSubtitles.com].txt', './data/Understanding Linguistics Noam Chomsky Talks at Google [English] [DownloadYoutubeSubtitles.com].txt', './data/Debate Noam Chomsky & Michel Foucault - On human nature [Subtitled] [English] [DownloadYoutubeSubtitles.com].txt', './data/Noam Chomsky Coronavirus — what is at stake DiEM25 TV [English (auto-generated)] [DownloadYoutubeSubtitles.com].txt', './data/Steven Pinker and Rebecca Newberger Goldstein The long reach of reason [English] [DownloadYoutubeSubtitles.com].txt', './data/Is the world getting better or worse A look at the numbers Steven Pinker [English (auto-generated)] [DownloadYoutubeSubtitles.com].txt', './data/RSA ANIMATE The Power of Networks [English (auto-generated)] [DownloadYoutubeSub

In [50]:
# store them in a list
path_txt_files = getFiles("./data")

# read the .txt files and save them in one 'text' variable
text = ''
for path_txt_file in path_txt_files:
    with open(path_txt_file, 'r') as f:
        text += f.read()
        # add a point and space an between each text
        text += '. '

In [51]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  3346688


In [52]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#$%&'(),-./0123456789:;>?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz} àçèéêíîïñóôöü​–—’“”…♪
107


In [53]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[65, 66, 66, 1, 77, 65, 62, 75, 62]
hii there


In [54]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([3346688]) torch.int64
tensor([65, 62, 69, 69, 72,  1, 73, 75, 72, 63, 62, 76, 76, 72, 75,  1, 60, 65,
        72, 70, 76, 68, 82,  1, 66, 77,  8, 76,  1, 58,  1, 66, 77,  8, 76,  1,
        58,  0,  0, 75, 62, 58, 69, 69, 82,  1, 58,  1, 73, 69, 62, 58, 76, 78,
        75, 62,  1, 77, 72,  1, 77, 72,  1, 70, 62, 62, 77,  1, 82, 72, 78,  1,
        58, 71, 61,  1, 58, 71,  0,  0, 65, 72, 71, 72, 75,  1, 77, 72,  1, 65,
        58, 79, 62,  1, 82, 72, 78,  1, 72, 71,  1, 77, 65, 66, 76,  1, 73, 72,
        61, 60, 58, 76, 77,  1, 76, 72,  0,  0, 80, 62, 69, 60, 72, 70, 62,  1,
        73, 69, 62, 58, 76, 78, 75, 62,  1, 77, 72,  1, 59, 62,  1, 80, 66, 77,
        65,  1, 82, 72, 78,  0,  0, 76, 72,  0,  0, 82, 72, 78, 75,  1, 80, 72,
        75, 68,  1, 66, 76,  1, 65, 78, 64, 62,  1, 58, 71, 61,  1, 82, 72, 78,
         1, 65, 58, 79, 62,  1, 73, 78, 59, 69, 66, 76, 65, 62, 61,  0,  0, 72,
        79, 62, 75,  1, 16, 15, 15,  1, 59, 72, 72, 68, 76,  1, 72, 71,  1, 70,
      

In [55]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [56]:
block_size = 8
train_data[:block_size+1]

tensor([65, 62, 69, 69, 72,  1, 73, 75, 72])

In [57]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([65]) the target: 62
when input is tensor([65, 62]) the target: 69
when input is tensor([65, 62, 69]) the target: 69
when input is tensor([65, 62, 69, 69]) the target: 72
when input is tensor([65, 62, 69, 69, 72]) the target: 1
when input is tensor([65, 62, 69, 69, 72,  1]) the target: 73
when input is tensor([65, 62, 69, 69, 72,  1, 73]) the target: 75
when input is tensor([65, 62, 69, 69, 72,  1, 73, 75]) the target: 72


In [58]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[77, 75, 82,  1, 58, 71, 61,  1],
        [64, 62,  1, 78, 71, 60, 62, 75],
        [ 0,  0, 77, 62, 76, 77, 76,  1],
        [63, 75, 72, 70,  1, 59, 75, 66]])
targets:
torch.Size([4, 8])
tensor([[75, 82,  1, 58, 71, 61,  1, 62],
        [62,  1, 78, 71, 60, 62, 75, 77],
        [ 0, 77, 62, 76, 77, 76,  1, 63],
        [75, 72, 70,  1, 59, 75, 66, 61]])
----
when input is [77] the target: 75
when input is [77, 75] the target: 82
when input is [77, 75, 82] the target: 1
when input is [77, 75, 82, 1] the target: 58
when input is [77, 75, 82, 1, 58] the target: 71
when input is [77, 75, 82, 1, 58, 71] the target: 61
when input is [77, 75, 82, 1, 58, 71, 61] the target: 1
when input is [77, 75, 82, 1, 58, 71, 61, 1] the target: 62
when input is [64] the target: 62
when input is [64, 62] the target: 1
when input is [64, 62, 1] the target: 78
when input is [64, 62, 1, 78] the target: 71
when input is [64, 62, 1, 78, 71] the target: 60
when input is [64, 6

In [59]:
print(xb) # our input to the transformer

tensor([[77, 75, 82,  1, 58, 71, 61,  1],
        [64, 62,  1, 78, 71, 60, 62, 75],
        [ 0,  0, 77, 62, 76, 77, 76,  1],
        [63, 75, 72, 70,  1, 59, 75, 66]])


In [60]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits of the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        
        # idx are both (B, T) tensors of intergers
        # we want to look up the logits for the next token for each token in the input
        logits = self.token_embedding_table(idx) # (B, T, C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # (B*T, C)
            targets = targets.view(-1) # (B*T,)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions for the next token
            logits, loss = self.forward(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution or take the most likely
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat([idx, idx_next], dim=-1) # (B, T+1)
            
        return idx
            
        
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)    
print(logits.shape)
print(loss)

torch.Size([32, 107])
tensor(4.7771, grad_fn=<NllLossBackward0>)


In [61]:
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


ó!"(xö—I$B’î'LOoyNx_eDY;a$%
BFx4)PQ–,éàxwóJç'odwcYDe”èOAvB J-​Hó…lB68s1j’!(!éeDeH​TFüam_jEK>El>h'$”o


In [62]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [63]:
batch_size = 32
for i in range(50000):
    xb, yb = get_batch('train')
    
    # evaluate the loss on the training data
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()  
    
    if i % 10000 == 0:
        print(f"step {i} train loss: {loss.item():.3f}")
        print(decode(m.generate(idx, max_new_tokens=30)[0].tolist()))

print("loss : ", loss.item())
  
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

step 0 train loss: 5.113

BElêRMZ[ZIèï
1ïxt9c>f6XHzà&Qeí


step 10000 train loss: 2.371



l ath
sit Ty s t
baryer w io
step 20000 train loss: 2.492

chomo mpr con p s to ingequs, 
step 30000 train loss: 2.348

thinonoustestheautr bscre-cind
step 40000 train loss: 2.571

yo usereraimo t ikisinses t g 
loss :  2.5135092735290527

est tha on hout t’êRYoco I
atoator?
sthe
Mys m t y insh 19415 m wserianchatheve woree
FORD:
baniora 
